# Produce probability-estimation-playground.jsonl

In [1]:
%%time
import sys
sys.path.append('../python/')

from trectools import TrecRun, TrecQrel
import pandas as pd
from tqdm import tqdm
from run_file_processing import load_all_runs, normalize_run
from evaluation_util import __adjust_qrels_to_pool
from probability_estimation import load_pool_task, CountProbabilityEstimator, RunIndependentCountProbabilityEstimator, PoissonEstimator
import json

RUNS = list(load_all_runs('../resources/processed/normalized-runs/trec-system-runs/trec13/robust').keys())

class PredictAlways0:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:1, 1:0, 2:0, 3:0}

class PredictAlways1:
    def estimate_probabilities(run, qrels, relevance_level):
        return {0:0, 1:1, 2:0, 3:0}

def persist_eval_input(run, qrels_incomplete, qrels_complete, out):
    run = normalize_run(run, 10).run_data
    
    for topic in tqdm(run['query'].unique()):
        run_for_topic = run[run['query'] == topic]
        
        out_obj = {
            'system': run_for_topic.iloc[0]['system'],
            'query': run_for_topic.iloc[0]['query'],
            'run': run_for_topic.to_json(lines=True, orient='records'),
            'qrels_incomplete': qrels_incomplete[qrels_incomplete['query'] == topic].to_json(lines=True, orient='records'),
            'qrels_complete': qrels_complete[qrels_complete['query'] == topic].to_json(lines=True, orient='records'),
        }
        
        df = run_for_topic.copy()
        df = pd.merge(df, qrels_complete[["query","docid","rel"]], how="left")
        df = df[~df["rel"].isnull()]
        
        if len(df) == 0:
            continue
        
        out.write(json.dumps(out_obj) + '\n')


Load runs: 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [00:05<00:00, 18.82it/s]

CPU times: user 6.23 s, sys: 714 ms, total: 6.94 s
Wall time: 6.43 s


In [ ]:
with open('probability-estimation-playground.jsonl', 'w') as f:
    for run_name in RUNS:
        run = normalize_run(TrecRun(run_name), 10)
        qrels = TrecQrel('../resources/unprocessed/topics-and-qrels/qrels.robust04.txt')
        task = {
            'working_directory': '../resources/',
            'trec_identifier': 'trec-system-runs-trec13-robust',
            'run': run_name.replace('../resources', 'src/main/resources')
        }
        pool = load_pool_task(task)
        pool = {k:v for k,v in pool.items() if 'depth-10-pool-incomplete-for' in k}
        assert len(pool) == 1
        pool = pool[list(pool.keys())[0]]

        qrels_incomplete = __adjust_qrels_to_pool(qrels, pool)

        persist_eval_input(run, qrels_incomplete.qrels_data.copy(), qrels.qrels_data.copy(), f)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:36<00:00,  6.83it/s]


In [5]:
print('Done')

Done
